## DIA 100: Permitir al Usuario Elegir el Formato del Reporte (PDF o CSV)

¡Felicidades por llegar al Día 100 de práctica continua! 🙌
Hoy vas a integrar una opción para que el usuario pueda elegir si desea su reporte en formato PDF o CSV al momento de exportarlo o recibirlo.

✅ ¿Qué aprenderás hoy?

📤 Manejar formularios con selección de formato.

🧾 Generar dinámicamente el archivo según elección.

📬 Preparar el archivo para descarga o envío.

🧩 1. Ruta: /descargar-reporte
Agrega esta ruta a tu archivo Flask:

python
Copiar
Editar
@app.route('/descargar-reporte', methods=['GET', 'POST'])
@login_required
def descargar_reporte():
    if request.method == 'POST':
        fecha_inicio = request.form['fecha_inicio']
        fecha_fin = request.form['fecha_fin']
        formato = request.form['formato']

        if not fecha_inicio or not fecha_fin:
            return "⚠️ Fechas requeridas.", 400

        f1 = datetime.strptime(fecha_inicio, '%Y-%m-%d')
        f2 = datetime.strptime(fecha_fin, '%Y-%m-%d')

        registros = Registro.query.filter(
            Registro.user_id == current_user.id,
            Registro.fecha >= f1,
            Registro.fecha <= f2
        ).order_by(Registro.fecha.desc()).all()

        if not registros:
            return "⚠️ No se encontraron datos.", 404

        if formato == 'csv':
            # CSV
            data = [{
                "Fecha": r.fecha.strftime('%Y-%m-%d %H:%M:%S'),
                "F1": r.f1, "F2": r.f2, "F3": r.f3, "F4": r.f4,
                "Predicción": r.prediccion
            } for r in registros]
            df = pd.DataFrame(data)
            output = io.StringIO()
            df.to_csv(output, index=False)
            output.seek(0)
            return send_file(io.BytesIO(output.getvalue().encode()),
                             mimetype='text/csv',
                             as_attachment=True,
                             download_name=f'reporte_{fecha_inicio}_a_{fecha_fin}.csv')

        elif formato == 'pdf':
            # PDF
            buffer = io.BytesIO()
            p = canvas.Canvas(buffer, pagesize=letter)
            width, height = letter

            p.setFont("Helvetica-Bold", 14)
            p.drawString(50, height - 50, f"Reporte de {current_user.username}")
            p.setFont("Helvetica", 12)
            p.drawString(50, height - 70, f"Desde {fecha_inicio} hasta {fecha_fin}")

            y = height - 100
            p.setFont("Helvetica-Bold", 10)
            p.drawString(50, y, "Fecha        | F1 | F2 | F3 | F4 | Predicción")
            y -= 15
            p.line(50, y, width - 50, y)
            y -= 15
            p.setFont("Helvetica", 9)

            for r in registros:
                linea = f"{r.fecha.strftime('%Y-%m-%d %H:%M')} | {r.f1} | {r.f2} | {r.f3} | {r.f4} | {r.prediccion}"
                p.drawString(50, y, linea)
                y -= 15
                if y < 50:
                    p.showPage()
                    y = height - 50

            p.save()
            buffer.seek(0)
            return send_file(buffer,
                             mimetype='application/pdf',
                             as_attachment=True,
                             download_name=f'reporte_{fecha_inicio}_a_{fecha_fin}.pdf')

        else:
            return "⚠️ Formato no válido", 400

    return render_template("descargar_reporte.html")
🖼️ 2. Template: descargar_reporte.html
html
Copiar
Editar
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Descargar Reporte</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">
<div class="container mt-5">
    <h2 class="text-center mb-4">📁 Descargar Reporte</h2>
    <form method="POST" class="row g-3">
        <div class="col-md-4">
            <label class="form-label">Desde:</label>
            <input type="date" name="fecha_inicio" class="form-control" required>
        </div>
        <div class="col-md-4">
            <label class="form-label">Hasta:</label>
            <input type="date" name="fecha_fin" class="form-control" required>
        </div>
        <div class="col-md-4">
            <label class="form-label">Formato:</label>
            <select name="formato" class="form-select" required>
                <option value="pdf">PDF</option>
                <option value="csv">CSV</option>
            </select>
        </div>
        <div class="col-12 text-end">
            <button type="submit" class="btn btn-primary mt-3">📤 Descargar</button>
        </div>
    </form>
</div>
</body>
</html>
